In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)

# 3-Node Capacity Expansion

This example builds on the [single-node capacity expansion example]() and extends it to three regions in Australia (New South Wales, Victoria, and South Australia), which can be connected via transmission links.
The purpose is to illustrate how to set up a capacity expansion model with multiple interconnected regions, where the model can optimize for trade-offs between generation capacity, storage, and transmission infrastructure.

In this first block, we import the necessary libraries and set up some basic functions (e.g. annuity factors) and constants (e.g. temporal resolution and solver):

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pypsa
from pypsa.common import annuity

RESOLUTION = 3  # 3-hourly
SOLVER = "highs"  # or 'gurobi'

In a fresh PyPSA network instance, we set the coordinates of each state's capital (Sydney, Melbourne, and Adelaide) as the coordinates of the electricity buses.

In [ ]:
n = pypsa.Network()

REGIONS = pd.Index(["VIC", "SA", "NSW"])
LAT = [-37.81, -34.93, -33.87]
LON = [145.01, 138.63, 151.20]
n.add("Bus", REGIONS, x=LON, y=LAT);

We also define a range of carriers (i.e. energy carriers or technologies) that we will use in the model. This will be useful later when plotting optimisation results.

In [ ]:
CARRIERS = {
    "solar": "gold",
    "wind": "steelblue",
    "load shedding": "indianred",
    "battery charger": "lightgreen",
    "battery discharger": "lightgreen",
    "battery storage": "grey",
    "battery": "grey",
    "electrolysis": "violet",
    "turbine": "violet",
    "hydrogen storage": "orchid",
    "hydrogen": "orchid",
    "AC": "black",
    "HVDC": "lightseagreen",
    "load": "slategrey",
}

n.add(
    "Carrier",
    CARRIERS.keys(),
    color=CARRIERS.values(),
);

As data, we have given:

- `australia-example-p_set.csv` - a time series of electricity demand in each region in 2011 from the [Australian Energy Market Operator (AEMO)](https://aemo.com.au/energy-systems/electricity/national-electricity-market-nem/data-nem/aggregated-data) 
- `australia-example-p_max_pu.csv` - a time series of capacity factors in 2011 for onshore wind and solar PV for each region from [model.energy](https://model.energy/)
- `australia-example-shapes.geojson` - a geographical representation of the three regions from the [GeoBoundaries](https://www.geoboundaries.org/) project

The timestamps in the datasets are given in [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time).

The datasets have been obtained from the following code:

```python
import pandas as pd
import geopandas as gpd

REGIONS = ["VIC", "SA", "NSW"]

# 1. Capacity factors
capacity_factors = {
    "VIC": "https://model.energy/data/time-series-c3ec7795e4d52617defed46495899b50.csv",
    "SA": "https://model.energy/data/time-series-efcb605660bd6f9db75fc7c5b3fd5379.csv",
    "NSW": "https://model.energy/data/time-series-ef16f3da1c24442d7efac9d7c4a0ce05.csv",
}

p_max_pu = pd.concat({
    c: pd.read_csv(
        url,
        index_col=0,
        parse_dates=True,
    )
    for c, url in capacity_factors.items()
}, axis=1)

p_max_pu.to_csv("australia-example-p_max_pu.csv")

# 2. Electricity demand
so = {'User-Agent': 'Mozilla/5.0'}
year_months = [f"2011{m:02d}" for m in range(1, 13)] + ["201201"]  # All months in 2011 + Jan 2012

p_set = pd.concat([
    pd.concat([
        pd.read_csv(
            f"https://aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_{ym}_{region}1.csv",
            storage_options=so,
            index_col=1,
            parse_dates=True
        )["TOTALDEMAND"].rename(region)
        for ym in year_months
    ])
    for region in REGIONS
], axis=1)

p_set = p_set.resample("1h").mean().ffill(inplace=True)

p_set = p_set.tz_localize(
    "Australia/NSW", ambiguous="NaT", nonexistent="shift_backward"
).tz_convert("UTC").tz_localize(None).reindex(index=p_max_pu.index)

p_set.to_csv("australia-example-p_set.csv")

# 3. Geographical shapes
url = "https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/bdfb316b1fdcac1473051979152cac0943c549fa/releaseData/gbOpen/AUS/ADM1/geoBoundaries-AUS-ADM1-all.zip"

gdf = gpd.read_file(url, layer="geoBoundaries-AUS-ADM1_simplified")
gdf.index = gdf.shapeISO.str.split("-").str[1]
gdf = gdf.loc[REGIONS]
gdf.to_file("australia-example-shapes.geojson", driver="GeoJSON")
```

We load this data and set the snapshots of the network to the timestamps of the time series data.

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/oPQbAebrciFBZP2/download/australia-example-p_max_pu.csv"
p_max_pu = pd.read_csv(url, index_col=0, parse_dates=True, header=[0, 1])
display(p_max_pu.head(3))

url = "https://tubcloud.tu-berlin.de/s/8C6d7z3HxE7yZi9/download/australia-example-p_set.csv"
p_set = pd.read_csv(url, index_col=0, parse_dates=True)
display(p_set.head(3))

n.set_snapshots(p_max_pu.index)

Then, we add the loads for each region:

In [ ]:
n.add("Load", REGIONS, suffix=" load", bus=REGIONS, p_set=p_set, carrier="load");

For the wind and solar generators, we pass the capacity factors as `p_max_pu` and add the annuitized capital costs. Here, we assume a capital cost of 2000 AUD/kW for wind and 700 AUD/kW for solar, with a discount rate of 5% and a lifetime of 20 years. Note again that the capacity is given in MW, so we multiply the capital costs by 1000 to convert them to kW.

As a trick for modelling curtailment, we add a small `marginal_cost` to the wind generators to nudge those to be curtailed first before solar.

In [ ]:
p_max_pu_wind = p_max_pu.xs("onwind", level=1, axis=1).rename(
    columns=lambda s: s + " wind"
)

n.add(
    "Generator",
    p_max_pu_wind.columns,
    bus=REGIONS,
    p_max_pu=p_max_pu_wind,
    p_nom_extendable=True,
    capital_cost=annuity(0.05, 30) * 2_000_000,
    marginal_cost=0.5,
    carrier="wind",
)

p_max_pu_solar = p_max_pu.xs("solar", level=1, axis=1).rename(
    columns=lambda s: s + " solar"
)

n.add(
    "Generator",
    p_max_pu_solar.columns,
    bus=REGIONS,
    p_max_pu=p_max_pu_solar,
    p_nom_extendable=True,
    capital_cost=annuity(0.05, 30) * 700_000,
    carrier="solar",
);

We also add a generator with a very high marginal cost to represent load shedding with a value of lost load (VoLL) of 3000 AUD/MWh.

In [ ]:
n.add(
    "Generator",
    REGIONS,
    suffix=" load shedding",
    bus=REGIONS,
    p_nom=p_set.max(),
    marginal_cost=3000,
    carrier="load shedding",
);

To save some computation time, we only sample every third snapshot, which corresponds to a temporal resolution of 3 hours. Note that the snapshot weightings have to be adjusted accordingly.

Let's give this basic model a first spin:

In [ ]:
n.set_snapshots(n.snapshots[::RESOLUTION])
n.snapshot_weightings.loc[:, :] = RESOLUTION

n.optimize(solver_name=SOLVER, log_to_console=False)

In this first run, every region is isolated (no transmission) and has no storage (no batteries or hydrogen storage).

Around 2.5% of the load is shed; roughly 50 TWh come from solar and 80 TWh from wind. The average cost of 226.57 AUD/MWh is quite high.

In [ ]:
display(n.statistics.energy_balance().div(1e6).round(2).sort_values())  # TWh

average_cost = (
    (n.statistics.capex().sum() + n.statistics.opex().sum())
    / n.loads_t.p_set.sum().sum()
    / RESOLUTION
)

display(f"Average cost: {average_cost:.2f} AUD/MWh")

The statistics and plotting functionality of PyPSA can be used to create a more detailed overview of the results on a map. For example, we can plot the energy balance and energy mix of each region as pie charts, where the upper half shows the generation and the lower half the consumption. We can also add other geographical layers from other objects to the map, such as the geographical shapes of the regions colored by the average locational marginal price (LMP).

In [ ]:
# Create an empty figure with a Cartopy projection
crs = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": crs})

# Use the energy balance statistics to prepare the bus sizes and plot the network
bus_size = n.statistics.energy_balance(groupby=["bus", "carrier"]).droplevel(
    "component"
)
n.plot(ax=ax, bus_size=bus_size / 4e7, margin=0.75, bus_split_circle=True)

# Load the shapefiles and add them to the plot colored by average LMP
prices = n.buses_t.marginal_price.mean()
gdf = gpd.read_file(
    "https://tubcloud.tu-berlin.de/s/4n9PegitETESBGR/download/australia-example-shapes.geojson"
).set_index("index")
gdf.to_crs(crs).plot(ax=ax, column=prices, cmap="RdYlGn_r", vmin=90, vmax=240)

# Add a legend for the LMP color scale
norm = plt.Normalize(vmin=90, vmax=240)
sm = plt.cm.ScalarMappable(cmap="RdYlGn_r", norm=norm)
fig.colorbar(sm, ax=ax, label="LMP [AUD/MWh]", shrink=0.4)

Now, we add some storage options to each model region:
- For **battery storage**, we separately model the battery inverter (charger and discharger) and the battery storage itself. This has the advantage that the power-to-energy ratio can be optimised.
- For **hydrogen storage**, we model the electrolyser, the hydrogen storage, and the turbine for re-electrification separately, so that these components can be independently sized.

In [ ]:
n.add("Bus", REGIONS, suffix=" hydrogen", carrier="hydrogen", x=LON, y=LAT)

n.add(
    "Link",
    REGIONS,
    suffix=" electrolysis",
    bus0=REGIONS,
    bus1=pd.Index(REGIONS) + " hydrogen",
    carrier="electrolysis",
    p_nom_extendable=True,
    efficiency=0.7,
    capital_cost=annuity(0.05, 25) * 2_500_000,
)

n.add(
    "Link",
    REGIONS,
    suffix=" turbine",
    bus0=pd.Index(REGIONS) + " hydrogen",
    bus1=REGIONS,
    carrier="turbine",
    p_nom_extendable=True,
    efficiency=0.4,
    capital_cost=annuity(0.05, 25) * 2_000_000,
)

n.add(
    "Store",
    REGIONS,
    suffix=" hydrogen storage",
    bus=pd.Index(REGIONS) + " hydrogen",
    carrier="hydrogen storage",
    capital_cost=annuity(0.05, 30) * 80,
    e_nom_extendable=True,
    e_cyclic=True,
)

n.add("Bus", REGIONS, suffix=" battery", carrier="battery", x=LON, y=LAT)

n.add(
    "Link",
    REGIONS,
    suffix=" battery charger",
    bus0=REGIONS,
    bus1=REGIONS + " battery",
    carrier="battery charger",
    p_nom_extendable=True,
    efficiency=0.95,
    capital_cost=annuity(0.05, 10) * 300_000,
)

n.add(
    "Link",
    REGIONS,
    suffix=" battery discharger",
    bus0=REGIONS + " battery",
    bus1=REGIONS,
    carrier="battery discharger",
    p_nom_extendable=True,
    efficiency=0.95,
)

n.add(
    "Store",
    REGIONS,
    suffix=" battery storage",
    bus=REGIONS + " battery",
    carrier="battery storage",
    capital_cost=annuity(0.05, 25) * 250_000,
    e_nom_extendable=True,
    e_cyclic=True,
);

For the battery storage, we need to take special care of the inverter. As the same component can be used for charging and discharging, the capacities of our battery charger and discharger component must be linked.
This is not done automatically by PyPSA, so we have to add an extra constraint, which we can pass as `extra_functionality` to the optimisation.

In [ ]:
def battery_constraint(n: pypsa.Network, sns: pd.Index) -> None:
    """Constraint to ensure that the nominal capacity of battery chargers and dischargers are in a fixed ratio."""
    dischargers_i = n.links[n.links.index.str.contains(" discharger")].index
    chargers_i = n.links[n.links.index.str.contains(" charger")].index

    eff = n.links.efficiency[dischargers_i].values
    lhs = n.model["Link-p_nom"].loc[chargers_i]
    rhs = n.model["Link-p_nom"].loc[dischargers_i] * eff

    n.model.add_constraints(lhs == rhs, name="Link-charger_ratio")


n.optimize(
    solver_name=SOLVER, log_to_console=False, extra_functionality=battery_constraint
)

A look at the new energy balance shows use of both battery and hydrogen storage, as well as a shift from wind to solar generation compared to the previous run.
In fact, overall, we see more generation from wind and solar than the load, in order to compansate for the losses in the storage systems.
There is a difference of around 5 TWh between power consumption and injection of the battery and a 15 TWh difference between electrolyser electricity consumption and turbine electricity production. Yet, overall, average costs are reduced drastically to 100.51 AUD/MWh.

In [ ]:
display(
    n.statistics.energy_balance(bus_carrier="AC").div(1e6).round(2).sort_values()
)  # TWh

average_cost = (
    (n.statistics.capex().sum() + n.statistics.opex().sum())
    / n.loads_t.p_set.sum().sum()
    / RESOLUTION
)

print(f"Average cost: {average_cost:.2f} AUD/MWh")

Again, we can show these results spatially disagreggated on a map. The upper and lower half circles are still equal because no imbalances are possible due to the lack of transmission links. However, we can now see green elements for battery and purple elements for hydrogen storage, in addition to wind and solar generation.

In [ ]:
bus_size = (
    n.statistics.energy_balance(groupby=["bus", "carrier"])
    .droplevel("component")
    .loc[REGIONS]
)

n.plot(bus_size=bus_size / 4e7, margin=0.75, bus_split_circle=True);

Of course, there are other plots we can create from the optimised model, for instance, a price duration curve for Victoria.
For this we sort the time series of bus marginal prices in descending order and reindex to a relative axis from 0% to 100%:

In [ ]:
pdc = (
    n.buses_t.marginal_price["VIC"].sort_values(ascending=False).reset_index(drop=True)
)
pdc.index = np.arange(0, 100, 100 / len(pdc.index))
pdc.plot(
    ylim=[0, 1000], xlim=[0, 100], xlabel="Share of time [%]", ylabel="Price [AUD/MWh]"
)

As a final modelling step, we will now add transmission links between the regions. By modelling the interconnectors as bidirectional links, we will assume that they are controllable point-to-point HVDC connections (rather than lines subject to Kirchhoff's Voltage Law). They are also assumed to be lossless, otherwise a similar workaround with two unidirectional links as for battery inverters has to be implemented. The cost of the transmission links is length-dependent and set to 1000 AUD/MW/km.

In [ ]:
connections = [
    ("VIC", "NSW", 700),  # km
    ("VIC", "SA", 650),  # km
    ("NSW", "SA", 1150),  # km
]

for bus0, bus1, length in connections:
    n.add(
        "Link",
        f"{bus0}-{bus1}",
        bus0=bus0,
        bus1=bus1,
        carrier="HVDC",
        p_nom_extendable=True,
        capital_cost=annuity(0.05, 40) * 1_000 * length,
        p_min_pu=-1,  # bidirectional
    )

In [ ]:
n.optimize(
    solver_name=SOLVER, log_to_console=False, extra_functionality=battery_constraint
)

With transmission enabled, we can see that the average costs are reduced further to 93.00 AUD/MWh.
This is achieved as the interconnection facilitates the integration of wind variability across regions, 
reducing curtailment and the some need for hydrogen storage. The energy mix also shifts slightly from solar towards wind.

In [ ]:
display(
    n.statistics.energy_balance(bus_carrier="AC").div(1e6).round(2).sort_values()
)  # TWh

average_cost = (
    (n.statistics.capex().sum() + n.statistics.opex().sum())
    / n.loads_t.p_set.sum().sum()
    / RESOLUTION
)

display(f"Average cost: {average_cost:.2f} AUD/MWh")

Now, we can add further elements to the map, such as the interconnector capacities, the net flow direction as well as the average loading of the interconnectors, which show how Victoria becomes a net importer of electricity from New South Wales and South Australia.

In [ ]:
bus_size = (
    n.statistics.energy_balance(bus_carrier="AC", groupby=["bus", "carrier"])
    .droplevel("component")
    .loc[REGIONS]
    .drop("HVDC", level="carrier")
)

link_flow = n.links_t.p0.mean()[n.links.carrier == "HVDC"]

link_width = n.links.p_nom_opt[n.links.carrier == "HVDC"]

link_loading = n.links_t.p0.abs().mean()[n.links.carrier == "HVDC"] / link_width

n.plot(
    bus_size=bus_size / 4e7,
    margin=0.75,
    bus_split_circle=True,
    link_flow=link_flow / 50,
    link_width=link_width / 1e3,
    link_color=link_loading,
)

Of course, we can also just retrieve the optimised interconnector capacities as `pandas.Series`.

In [ ]:
n.links.query("carrier == 'HVDC'").p_nom_opt.round(2)

A further interesting insight reveals itself when we plto the interconnector flows over the year (here as weekly rolling averages).
The seasonality of the link between Victoria and New South Wales is striking, with VIC exporting to NSW in winter and importing in summer (Southern Hemisphere!).

In [ ]:
n.links_t.p0.loc[:, n.links.carrier == "HVDC"].rolling("7d").mean().plot(
    ylim=[-2500, 2500], grid=True
)

Also the state of charge profile of the hydrogen storage shows a clear seasonal pattern, with the storage being full in summer and empty in winter (Southern Hemisphere!).

In [ ]:
n.stores_t.e.filter(like="hydrogen").sum(axis=1).div(1e6).plot(
    ylabel="Hydrogen storage [TWh]", grid=True
)

Finally, there are some more functions to interactively plot hourly energy balance time series for different carriers, such as electricity:

In [ ]:
n.statistics.energy_balance.iplot.area(bus_carrier="AC")